**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-v94blq73
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-v94blq73
  Resolved https://github.com/Kaggle/learntools.git to commit 69bc6daec79619690e758841dc2df35708d226c8
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=b4e67719bb8b24e11df19739ac3dc86df8321b05440863e4e673f2a16823a9b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-vprzn635/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,90b5366f38e43094c5d0cebd1e492edf83bac474,d80bc0dd36a85306ac35402f5f62ff5f6775e93b06bda0...,2017-10-05 10:30:00+00:00,2017-10-05 10:45:00+00:00,1080,1.2,NaN,NaN,NaN,NaN,...,0.0,9.50,Cash,Dispatch Taxi Affiliation,NaN,NaN,None,NaN,NaN,None
1,d0dcdc1363e23f50594bb1202e1aa719af4fdf2b,4e04293a1ab799479725ae45fabd3c04bb33c9e46ad0de...,2017-09-26 22:30:00+00:00,2017-09-26 22:30:00+00:00,360,1.4,NaN,NaN,NaN,NaN,...,1.0,8.00,Cash,Dispatch Taxi Affiliation,NaN,NaN,None,NaN,NaN,None
2,33b6c9683fbdbf29970b8145b6d82ae0f7cf168c,3fb54b9070b703bd28358e704311d7fe8afb6950881654...,2017-09-27 12:15:00+00:00,2017-09-27 13:00:00+00:00,2340,18.0,NaN,NaN,NaN,NaN,...,5.0,63.60,Credit Card,KOAM Taxi Association,NaN,NaN,None,NaN,NaN,None
3,55d64e49c22169aa5b47ad5310744ae522addfb2,4097b390ef36dd0909677975a8e6237b7a6947dfc23e24...,2016-04-05 21:30:00+00:00,2016-04-05 21:30:00+00:00,222,1.4,NaN,NaN,NaN,NaN,...,0.0,5.00,Cash,Metro Group,NaN,NaN,None,NaN,NaN,None
4,8c7542e9342b23da2dc382a7a11b305292e7824f,7fb7fcd6d964e468d888284215533053c6b7aa59b72cda...,2017-09-27 18:30:00+00:00,2017-09-27 18:30:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,3.25,Cash,Choice Taxi Association,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [5]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-09,98175.571429
1,2016-02-10,91321.142857
2,2016-03-02,91927.000000
3,2016-01-19,85156.571429
4,2016-01-16,84642.285714


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [10]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,27.0,2013-10-03 08:15:00+00:00,2013-10-03 08:15:00+00:00,1
1,27.0,2013-10-03 10:45:00+00:00,2013-10-03 10:45:00+00:00,2
2,27.0,2013-10-03 11:00:00+00:00,2013-10-03 11:00:00+00:00,3
3,27.0,2013-10-03 11:00:00+00:00,2013-10-03 11:00:00+00:00,3
4,27.0,2013-10-03 12:15:00+00:00,2013-10-03 12:15:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [13]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                          trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,01e6dff46957e9c42a598928efa3e4e88522c1696269b4...,2013-10-03 02:30:00+00:00,2013-10-03 02:30:00+00:00,165.0
1,01e6dff46957e9c42a598928efa3e4e88522c1696269b4...,2013-10-03 05:30:00+00:00,2013-10-03 05:00:00+00:00,180.0
2,01e6dff46957e9c42a598928efa3e4e88522c1696269b4...,2013-10-03 18:00:00+00:00,2013-10-03 17:30:00+00:00,540.0
3,01e6dff46957e9c42a598928efa3e4e88522c1696269b4...,2013-10-03 23:45:00+00:00,2013-10-03 23:45:00+00:00,210.0
4,07e169f0700af9d3bbda767ee8adc7517e9c72beade49b...,2013-10-03 18:30:00+00:00,2013-10-03 18:45:00+00:00,1020.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [14]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*